<a href="https://colab.research.google.com/github/ragul-n/Gender-bias-in-Indian-cinema/blob/master/word2vec_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pysrt
!pip install gensim


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 104 kB 31.7 MB/s 
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13443 sha256=472ceea6373b5ca5e042c58e418730b86cfbed5f48e6987950e340be846375e4
  Stored in directory: /root/.cache/pip/wheels/30/a6/ab/4705174e11f44e74d58c14b32edbacbc852644f86658316aef
Successfully built pysrt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np 
import pysrt
import os 
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim 
from gensim.parsing.preprocessing import preprocess_documents
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:


class SentenceGenerator():
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):

            replace_dict={
                "\n":". ",  
                "?.":"?", 
                "...":" ",
                ",.":",",
                "..":". ",
                "!.":"! "
            }

            #.replace("\n",".").replace("?.","? ")..replace("..."," ")

            text=pysrt.open(os.path.join(self.dirname, fname)).text.lower()

            for i in replace_dict.keys():
                text=text.replace(i,replace_dict[i])

            for sent in preprocess_documents(sent_tokenize(text)):
                if len(sent)<5:
                    continue
                yield sent

sentences=SentenceGenerator("/content/drive/MyDrive/Dataset/subtitles")

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec



class MonitorCallback(CallbackAny2Vec):
    def on_epoch_end(self, model):
        print("Model loss:", model.get_latest_training_loss())  # print loss
        model.save('/content/drive/MyDrive/Dataset/word2vec/subtitles')

monitor = MonitorCallback()

model = gensim.models.Word2Vec(sentences, size=300, window=6, min_count=10, workers=4, iter = 10, callbacks=[monitor])
